# Deploying a Fine-tuned Vision Language Model to Amazon SageMaker AI for Inference

This notebook guides you through the process of deploying a fine-tuned Vision Language Model (VLM) to Amazon SageMaker. The deployment process includes several key steps:

1. **Environment Setup**: Installing necessary dependencies and configuring the AWS environment
2. **Model Artifact Management**: Locating and selecting the fine-tuned model artifacts from S3
3. **Container Infrastructure**: Building and pushing a custom container to Amazon ECR
4. **SageMaker Deployment**: Creating and deploying a SageMaker endpoint

**Prerequisites**

Before starting, ensure you have:
- AWS credentials configured with appropriate permissions
- AWS CLI installed
- Access to the S3 bucket containing your model artifacts

**Important Notes**

- The deployment uses an ml.g5.2xlarge instance which provides GPU acceleration necessary for efficient inference
- The custom container includes SWIFT framework for model serving

## Environment Setup

First, we'll install `jq`, a lightweight command-line JSON processor. This will be used to parse AWS metadata and credentials later in our deployment process.

In [ ]:
!sudo apt-get install -qq -y jq > /dev/null

In [ ]:
%pip install boto3 sagemaker pandas huggingface_hub --quiet

In [ ]:
%load_ext autoreload
%autoreload 2

Import the required libraries:

In [ ]:
import json
import os
import boto3
import sagemaker
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from sagemaker.model import Model
from utils.model_manager import list_available_models

## Initialize AWS Services

In [ ]:
bucket_name = sagemaker.Session().default_bucket()
region = boto3.session.Session().region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]
role = sagemaker.get_execution_role()
session = sagemaker.Session(boto_session=boto3.Session(region_name=region))
sm_client = boto3.client("sagemaker", region_name=region)

## List Available Model Artifacts

List the available models based on S3 key prefix and "model.tar.gz".

In [ ]:
model_name = "finetune-"

df = list_available_models(bucket_name, model_name)
df

In [ ]:
# Set up the S3 URI from which we will download the model
model_key=df['Key'].iloc[0]
s3_model_uri = f"s3://{bucket_name}/{model_key}"
print(f"Selected model for deployment: {model_key}")
print(f"S3 Model URI: {s3_model_uri}")

## Build and Push Docker Inference Container

Amazon SageMaker AI offers [three primary methods](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers.html) for deploying ML models to an SageMaker AI Inference Endpoint:
1. Using pre-built SageMaker containers for standard frameworks like PyTorch or TensorFlow
2. Modifying existing Docker containers with your own dependencies through a requirements.txt file
3. Or creating completely custom containers that implements a web server listening for requests (/invocations) for maximum flexibility and control over dependencies and requirements.

To run the fine-tuned models you will build our custom container and push it to Amazon Elastic Container Registry (ECR). The container includes:
- SWIFT framework for model serving
- VLLM for high-throughput and memory efficient inference
- Required dependencies for the Qwen2-VL model
- API endpoint modifications for SageMaker integration


The container will be built from our Dockerfile and pushed to ECR, making it available for SageMaker to use when deploying our endpoint.

### Docker Installation

To create our custom container for model serving, we first need Docker installed in our environment. This script handles the installation of Docker and its dependencies, including necessary security keys and repository configurations.

**Install docker-cli**

At the end of this install you should see,

```bash
Client: Docker Engine - Community
 Version:           20.10.24
 API version:       1.41
 Go version:        go1.19.7
 Git commit:        297e128
 Built:             Tue Apr  4 18:21:03 2023
 OS/Arch:           linux/amd64
 Context:           default
 Experimental:      true
```

In [ ]:
!bash docker-artifacts/01_docker_install.sh

### Build and push a custom image

SageMaker Inference supports simplified deployment of Qwen2 using Large Model Inference (LMI) container images as indicated [here](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-container-docs.html) and available images listed [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md).

However, video inference is not a supported by vLLM/OpenAI api server out of the box and so we design a custom image with a custom inference handler adapter from vllm's [api_server.py](https://github.com/vllm-project/vllm/blob/main/vllm/entrypoints/api_server.py) to include video inference tag. We can build a custom image inside SageMaker Studio using `docker-cli`.

**Build our custom Docker image containing custom inference handler and push it to Amazon ECR (Elastic Container Registry).**

In [ ]:
REPO_NAME = "swift-json-vlm-container-finetuned"
os.environ['REPO_NAME'] = REPO_NAME
os.environ["S3_MODEL_URI"]=s3_model_uri

In [ ]:
%%bash -s {region} {account_id}

REGION=$1

VERSION_TAG="latest"
CURRENT_ACCOUNT_NUMBER=$2

echo "bash 02_build_and_push.sh $REPO_NAME $VERSION_TAG $REGION $CURRENT_ACCOUNT_NUMBER"
cd docker-artifacts && bash 02_build_and_push.sh $REPO_NAME $VERSION_TAG $REGION $CURRENT_ACCOUNT_NUMBER

### Getting Container Image URI

Retrieve the full URI of our Docker image from ECR. This URI is essential for SageMaker deployment as it tells SageMaker exactly where to find our custom container image. The URI follows the format:
`{account_id}.dkr.ecr.{region}.amazonaws.com/{repository_name}:{tag}`

In [ ]:
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{REPO_NAME}:latest"
print(f"Base image to deploy a SageMaker endpoint: {image_uri}")

os.environ['CUSTOM_IMAGE'] = image_uri

## Understanding the Model Serving Architecture

When we deploy our model to a SageMaker endpoint, here's how the components work together:

1. **Docker Container Structure**:
   - The container runs on the SageMaker instance (ml.g5.2xlarge)
   - The `swift deploy` command in our ENTRYPOINT starts a server that:
     - Loads the model into GPU memory
     - Sets up API endpoints for inference
     - Listens on port 8080 for requests

2. **Request Flow**:
   - External requests → SageMaker endpoint → Container's port 8080
   - The `sed` commands we used modified the API paths to match SageMaker's expected structure:
     - `/ping` for health checks
     - `/invocations` for model inference
     - `/invocations/completions` for completion requests

3. **SWIFT Framework's Role**:
   - Handles model loading and optimization with vLLM
   - Manages inference requests
   - Provides OpenAI-compatible API interface
   - Handles input preprocessing and output formatting

4. **SageMaker Integration**:
   - Routes HTTPS requests to our container
   - Manages container lifecycle
   - Handles authentication and scaling
   - Monitors container health via the `/ping` endpoint

This setup allows us to serve our fine-tuned model with production-grade reliability and performance.

**[Optional] We can run our container interactively in Terminal by using the command below. Make sure you are using a GPU instance for your Jupyterlab space since SWIFT inference requires a GPU.**

In [ ]:
# %%writefile run_container.sh
# # Get credentials from instance metadata
# export $(curl -s 169.254.170.2$AWS_CONTAINER_CREDENTIALS_RELATIVE_URI | jq -r '"AWS_ACCESS_KEY_ID="+.AccessKeyId, "AWS_SECRET_ACCESS_KEY="+.SecretAccessKey, "AWS_SESSION_TOKEN="+.Token')

# # Now run your docker container with these environment variables
# # Add --entrypoint /bin/bash in case you want to manually look into the container
# set -x
# docker run  --gpus all --network sagemaker -it -e AWS_ACCESS_KEY_ID -e AWS_SECRET_ACCESS_KEY -e AWS_SESSION_TOKEN \
#  -e ADAPTER_URI=$S3_MODEL_URI $REPO_NAME

# # docker run --network sagemaker -it -e AWS_ACCESS_KEY_ID -e AWS_SECRET_ACCESS_KEY -e AWS_SESSION_TOKEN \
# #  -e ADAPTER_URI=$S3_MODEL_URI $REPO_NAME

In [ ]:
# !bash run_container.sh

## Creating a SageMaker Model and deploy a SageMaker endpoint

Finally, we'll create a SageMaker model and deploy it to an inference endpoint. This will give us an HTTPS endpoint that we can use for inference.

Note: We're using an ml.g5.2xlarge instance which provides GPU acceleration necessary for efficient inference with a small multimodal model.

For more throughphut, lower latency, or when deploying a bigger model you might want to use a bigger instance type.

In [ ]:
sm_model_name = "document-to-json"
sm_endpoint_name = "document-to-json"

print(f"Model name: {sm_model_name}")
print(f"Endpoint name: {sm_model_name}")

Deploy our model to a SageMaker endpoint using an ml.g5.2xlarge instance. This GPU-enabled instance type provides the computational power needed for efficient inference with a Qwen2-VL model. The deployment:
- Creates a SageMaker model using our custom container
- Configures the endpoint with specified resources
- Initiates asynchronous deployment (wait=False)
- Sets up HTTPS endpoint for inference



In [ ]:
# Main deployment logic
from utils.helpers import check_model_exists, check_endpoint_config_exists, check_endpoint_exists, delete_all_resources

endpoint_exists = check_endpoint_exists(endpoint_name=sm_endpoint_name, sm_client=sm_client)
model_exists = check_model_exists(sm_model_name, sm_client=sm_client)
config_exists = check_endpoint_config_exists(sm_endpoint_name, sm_client=sm_client)

if endpoint_exists or model_exists or config_exists:
    print(f"\nFound existing resources:")
    if endpoint_exists:
        print(f"- Endpoint: {sm_endpoint_name}")
    if model_exists:
        print(f"- Model: {sm_model_name}")
    if config_exists:
        print(f"- Endpoint config: {sm_endpoint_name}")
    
    delete_all_resources(sm_model_name, sm_endpoint_name, sm_client=sm_client)

# Define environment variables for the model
environment = {
    "USE_HF_TRANSFER": "true",  # Enable faster downloads
    "HF_HUB_ENABLE_HF_TRANSFER": "1",
    "SIZE_FACTOR": "8",
    "MAX_PIXELS": "602112",
    "ADAPTER_URI": s3_model_uri,
    "SM_VLLM_SERVED_MODEL_NAME": sm_model_name, # you can name your model whatever you want
    "SM_VLLM_LIMIT_MM_PER_PROMPT": "image=2, video=0", # max number of images allowed in prompt. Increase for multi-page documents. Requires more memory.
    "SM_VLLM_MAX_NUM_SEQS":"8", # decrease if less GPU memory available
    "SM_VLLM_MAX_MODEL_LEN":"38608", # max context length, decrease if less GPU memory available
    "SM_VLLM_DTYPE": "bfloat16"
}

# If we get here, either nothing existed or we've cleaned up
model = Model(
    image_uri=image_uri,
    role=role,
    sagemaker_session=session,
    name=sm_model_name,
    env=environment,
)

print(f"\nEndpoint is now being deployed.... This may take several minutes.")

# Deploy a new endpoint
model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name=sm_endpoint_name,
    wait=True
)


## Next steps

After deploying the model as a SageMaker endpoint, we can call the model endpoint to run inference with the sample code in the next notebook [07_consume_model.ipynb](./07_consume_model.ipynb).